**Emilia Stefanowska, listopad 2022**

## Generowanie dyktand melodycznych - silnik aplikacji
*Generating melodic dictations - application engine*

In [1]:
import os
import pickle
from music21 import converter, chord, note, key, interval, pitch
import numpy as np
from keras.utils import np_utils
from keras.layers import LSTM, Input, Dropout, Dense, Embedding, Concatenate
from keras.models import Model
from keras.optimizers import RMSprop
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from datetime import datetime

Using TensorFlow backend.


**Ustawianie parametrów i folderu z zapisem danych** <br>
*Setting the parameters and backup folders*

In [2]:
date = str(datetime.now().strftime("%m-%d-%Y_%H-%M"))
training_id = '3-4_{}'.format(date) # 3-4 is time signature, and then time of training the network

# source_folder = 'training_set/midi/'
source_folder = 'MIDI_3-4/'
# source_folder = 'Sorted_BACH/3-4/'
backup_folder = 'backup/{}/'.format(training_id)

files = os.listdir(source_folder)

# Creating backup folder if doesn't already exist
if not os.path.exists('backup'):
    os.mkdir('backup')

# Creating backup folder for this specific training session
if not os.path.exists(backup_folder):
    os.mkdir(backup_folder)
    os.mkdir(os.path.join(backup_folder, 'training_set_backup'))
    os.mkdir(os.path.join(backup_folder, 'model'))
    
backup_training_set_folder = os.path.join(backup_folder, 'training_set_backup')
model_folder = os.path.join(backup_folder, 'model')

print('%s training set files in total' % len(files))

50 training set files in total


In [3]:
# Model parameters
state = 'read_and_save' # 'load'
seq_len = 12

In [4]:
pitch_read = [] # training set pitch matrix
rhythm_read = [] # training set rhythm matrix

**Odczytywanie i zapisywanie informacji ze zbioru treningowego** <br>
*Reading and saving data from the training set*

In [5]:
# If you want to read new pieces in the training set
if state == 'read_and_save':
    for file in files:
        dictation = converter.parse(source_folder + file)
        
        # Transpose to C-major
        dictation_key_signature = dictation.analyze('key')
        if (dictation_key_signature != key.Key('C')):
            i = interval.Interval(dictation_key_signature.tonic, pitch.Pitch('C'))
            dictation = dictation.transpose(i).transpose('P8')
            
        # Adding starting symbols to mark beginning of the piece
        pitch_read.extend(['S'] * seq_len)
        rhythm_read.extend([0.0] * seq_len)
        
        part_stream = dictation.parts.stream()
        my_part = part_stream[0] # ensuring we will use only the top melody 
        
        for element in my_part.flat.notesAndRests:         
            if element.isNote:
                pitch_read.append(str(element.nameWithOctave))
                rhythm_read.append(element.duration.quarterLength)

            if element.isRest:
                pitch_read.append(str(element.name))  # 'rest'
                rhythm_read.append(element.duration.quarterLength)
                
            if isinstance(element, chord.Chord): # if chord insert only the highest note
                pitch_read.append(element.pitches[-1].nameWithOctave)
                rhythm_read.append(element.duration.quarterLength)

    # Saving training set pitch and rhyhtm matrices into binary files
    with open(os.path.join(backup_training_set_folder, 'pitch'), 'wb') as file:
        pickle.dump(pitch_read, file)
    with open(os.path.join(backup_training_set_folder, 'rhythm'), 'wb') as file:
        pickle.dump(rhythm_read, file)
    
    # Create dictionaries with unique pitch and rhythm values and saving into binary files
    pitch_sorted = sorted(set(pitch_read))
    pitch_dict = dict((number, pitch) for pitch, number in enumerate(pitch_sorted))
    pitch_dict_reveresed = dict((pitch, number) for pitch, number in enumerate(pitch_sorted))
    
    rhythm_sorted = sorted(set(rhythm_read))
    rhythm_dict = dict((number, rhythm) for rhythm, number in enumerate(rhythm_sorted))
    rhythm_dict_reveresed = dict((rhythm, number) for rhythm, number in enumerate(rhythm_sorted))
    dict_tables = [pitch_sorted, pitch_dict, rhythm_sorted, rhythm_dict, pitch_dict_reveresed, rhythm_dict_reveresed]
    
    with open(os.path.join(backup_training_set_folder, 'dictionaries'), 'wb') as file:
        pickle.dump(dict_tables, file)

**Ładowanie wcześniej zapisanych danych** <br>
*Loading previously saved data*

In [6]:
if state == 'load':
    with open(os.path.join(backup_training_set_folder, 'pitch'), 'rb') as file:
        pitch_read = pickle.load(file)
    with open(os.path.join(backup_training_set_folder, 'rhythm'), 'rb') as file:
        rhythm_read = pickle.load(file)
    with open(os.path.join(backup_training_set_folder, 'dictionaries'), 'rb') as file:
        dict_tables = pickle.load(file)

**Przygotowywanie danych do sieci neuronowej** <br>
*Preparing data for neural network*

In [7]:
# We need to prepare data in pitch and rhythm matrices 
# that contain notes sequences of seq_len number of notes

pitch_input = []
pitch_output = []
rhythm_input = []
rhythm_output = []

for i in range(len(pitch_read) - seq_len):
    pitch_seq_in = pitch_read[i : i + seq_len] # seq_len number of pitches
    pitch_seq_out = pitch_read[i + seq_len] # the next note after the sequence of pitch_seq_in   
    rhythm_seq_in = rhythm_read[i : i + seq_len] # seq_len number of rhythms
    rhythm_seq_out = rhythm_read[i + seq_len] # the next note after the sequence of rhythm_seq_in
    
    # translating into dictionaries 
    pitch_input.append([pitch_dict[pitch_name] for pitch_name in pitch_seq_in])
    pitch_output.append(pitch_dict[pitch_seq_out])   
    rhythm_input.append([rhythm_dict[rhythm_name] for rhythm_name in rhythm_seq_in])
    rhythm_output.append(rhythm_dict[rhythm_seq_out])

number_of_sequences = len(pitch_input) # useful in reshaping matrices

# training input of the network
pitch_input = np.reshape(pitch_input, (number_of_sequences, seq_len))
rhythm_input = np.reshape(rhythm_input, (number_of_sequences, seq_len))
network_in = [pitch_input, rhythm_input] 

# training output of the network
pitch_output = np_utils.to_categorical(pitch_output, num_classes=len(pitch_dict)) # one-hot encoding *kodowanie 1 z n*
rhythm_output = np_utils.to_categorical(rhythm_output, num_classes=len(rhythm_dict)) # one-hot encoding *kodowanie 1 z n*
network_out = [pitch_output, rhythm_output] 

**Budowanie modelu sieci neuronowej** <br>
*Building the neural network*

//Buduję architekturę modelu za pomocą funkcjonalnego API Keras - przydaje się w mechaniźmie uwagi

In [8]:
# Network hyperparameters
embedding_size = 120 # 100
rnn_units = 300 # 256

# Useful training set variables
rhythm_size = len(rhythm_dict)
pitch_size = len(pitch_dict)

# Building the architecture
pitch_in = Input(shape = (None,))
rhythm_in = Input(shape = (None,))

x1 = Embedding(pitch_size, embedding_size)(pitch_in)
x2 = Embedding(rhythm_size, embedding_size)(rhythm_in) 

x = Concatenate()([x1,x2])

x = LSTM(rnn_units, return_sequences=True)(x) # First LSTM layer
x = Dropout(0.25)(x) 

x = LSTM(rnn_units)(x) # Second LSTM layer
x = Dropout(0.25)(x)
                                    
pitch_out = Dense(pitch_size, activation = 'softmax', name = 'pitch_output')(x)
rhythm_out = Dense(rhythm_size, activation = 'softmax', name = 'rhythm_output')(x)
   
model = Model([pitch_in, rhythm_in], [pitch_out, rhythm_out])

opti = RMSprop(lr = 0.001)
model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)


W1115 14:22:19.375891 21384 deprecation_wrapper.py:119] From C:\Users\stefa\.conda\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1115 14:22:19.388357 21384 deprecation_wrapper.py:119] From C:\Users\stefa\.conda\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1115 14:22:19.392981 21384 deprecation_wrapper.py:119] From C:\Users\stefa\.conda\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1115 14:22:19.691483 21384 deprecation_wrapper.py:119] From C:\Users\stefa\.conda\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default inst

**Zapisywanie schematu modelu i pokazanie podsumowania** <br>
*Save the plotted model and show summary*

In [9]:
plot_model(model, to_file=os.path.join(model_folder, 'model.png'), show_shapes = True, show_layer_names = True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 120)    4560        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 120)    1080        input_2[0][0]                    
__________________________________________________________________________________________________
concatenat

**Stworzenie mechanizmu zapisywania poprawienia wag i trenowanie modelu** <br>
*Create weights saving mechanism and train the model*

In [10]:
number_of_epochs = 2000
size_of_batch = 15

model_checkpoint = ModelCheckpoint(
    os.path.join(model_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 25
)


callbacks_list = [model_checkpoint, early_stopping]

model.save_weights(os.path.join(model_folder, "weights.h5"))

# Train the model
model.fit(network_in, network_out
          , epochs=number_of_epochs, batch_size=size_of_batch
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )


W1115 14:22:20.673196 21384 deprecation.py:323] From C:\Users\stefa\.conda\envs\generative\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3083 samples, validate on 771 samples
Epoch 1/2000
3083/3083 [==============================] - 11s 3ms/step - loss: 3.8955 - pitch_output_loss: 2.5630 - rhythm_output_loss: 1.3325 - val_loss: 3.5816 - val_pitch_output_loss: 2.5180 - val_rhythm_output_loss: 1.0636
Epoch 2/2000
3083/3083 [==============================] - 9s 3ms/step - loss: 3.3024 - pitch_output_loss: 2.2396 - rhythm_output_loss: 1.0628 - val_loss: 3.2952 - val_pitch_output_loss: 2.3857 - val_rhythm_output_loss: 0.9094
Epoch 3/2000
3083/3083 [==============================] - 10s 3ms/step - loss: 2.9983 - pitch_output_loss: 2.0726 - rhythm_output_loss: 0.9256 - val_loss: 3.0728 - val_pitch_output_loss: 2.2714 - val_rhythm_output_loss: 0.8015
Epoch 4/2000
3083/3083 [==============================] - 12s 4ms/step - loss: 2.7982 - pitch_output_loss: 1.9640 - rhythm_output_loss: 0.8342 - val_loss: 3.0738 - val_pitch_output_loss: 2.2715 - val_rhythm_output_loss: 0.8024
Epoch 5/2000
3083/3083 [======================

Epoch 37/2000
3083/3083 [==============================] - 13s 4ms/step - loss: 0.1396 - pitch_output_loss: 0.0960 - rhythm_output_loss: 0.0436 - val_loss: 5.9651 - val_pitch_output_loss: 4.0744 - val_rhythm_output_loss: 1.8906
Epoch 38/2000
3083/3083 [==============================] - 15s 5ms/step - loss: 0.1587 - pitch_output_loss: 0.1000 - rhythm_output_loss: 0.0587 - val_loss: 6.0081 - val_pitch_output_loss: 4.1310 - val_rhythm_output_loss: 1.8771
Epoch 39/2000
3083/3083 [==============================] - 13s 4ms/step - loss: 0.1593 - pitch_output_loss: 0.1028 - rhythm_output_loss: 0.0565 - val_loss: 6.0577 - val_pitch_output_loss: 4.1945 - val_rhythm_output_loss: 1.8631
Epoch 40/2000
3083/3083 [==============================] - 13s 4ms/step - loss: 0.1504 - pitch_output_loss: 0.0979 - rhythm_output_loss: 0.0524 - val_loss: 6.1803 - val_pitch_output_loss: 4.2730 - val_rhythm_output_loss: 1.9073
Epoch 41/2000
3083/3083 [==============================] - 13s 4ms/step - loss: 0.1339 -

KeyboardInterrupt: 

In [11]:
model.save(os.path.join(model_folder, "my_model"))